In [2]:
#install simple_salesforce module
!pip install pymysql

In [3]:
#Import Dependencies
import pandas as pd
import json
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
#Loading CSV files
Arl=pd.read_csv("ModifiedOutput_Crime_Arlington.csv")
Bet=pd.read_csv("ModifiedOutput_Crime_Bethesda.csv")
DC=pd.read_csv("ModifiedOutput_Crime_DC.csv")

In [ ]:
#Adding City label to the data

Arl["City"]="Arlington,VA"
Bet["City"]="Bethesda, MD"
DC["City"]="Washington,D.C"


In [ ]:
#Limiting the data to 2019

Arl = Arl.loc[Arl["Year"] ==2019, :]
Bet = Bet.loc[Bet["Year"] ==2019, :]
DC = DC.loc[DC["Year"] ==2019, :]


In [ ]:
#Changing column names (Latitude, Longtitude)

Arl=Arl.rename(columns={"Latitude (Arlington)": "Latitude", "Longitude (Arlington)": "Longitude"})
Bet=Bet.rename(columns={"Latitude (Bethesda)": "Latitude", "Longitude (Bethesda)": "Longitude"})
DC=DC.rename(columns={"Latitude (D.C.)": "Latitude", "Longitude (D.C.)": "Longitude"})


In [ ]:
#Droping extra columns

Arl=Arl.drop(['Unnamed: 0'], axis=1)
Bet=Bet.drop(['Unnamed: 0'], axis=1)
DC=DC.drop(['Unnamed: 0'], axis=1)

In [ ]:
Bet.head()


In [ ]:
#combining all data

Combined=pd.concat([Arl, Bet, DC])

#Exporting data to csv file
Combined.to_csv("CombinedData.csv")



In [ ]:
#Check Combined Dataframe
combined=pd.read_csv("CombinedData.csv", index_col=0)
combined.head()

In [ ]:
#Import configuration variables
# import sys
# sys.path.append('../../../../')
from config import sf_grant_type, sf_client_id, sf_client_secret, sf_username, sf_password, sf_security_token, sf_endpoint


In [ ]:
#set up salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [ ]:
#convert df to a list of dictionaries to help with bulk insert
combined_load = combined.to_dict('records')

In [ ]:
#Insert rows individually by looping throught the DF
for index, row in combined.iterrows():

    record = {
        'Crime Type': row['CrimeType'],
        'Year': row['Year'],
        'Latitude': 'Latitude',
        'Longitude': 'Longitude',
        'City':'City'
    }
    
    try:
        sf.Lead.create(record)
    except Exception as e:
        print(e)

In [ ]:
#inserting rows in bulk
try:
    sf.bulk.Lead.insert(combined_load)
except Exception as e:
    print(e)